Берем отызывы за лето (из архива с материалами или предыдущего занятия)
1. Учим conv сеть для классификации
2. Рассмотреть 2-а варианта сеточек
2.1 Инициализировать tf.keras.layers.Embedding предобученными векторами взять к примеру с https://rusvectores.org/ru/
2.2 Инициализировать слой tf.keras.layers.Embedding по умолчанию (ну то есть вам ничего не делать с весами)

Сравнить две архитектуры с предобученными весами и когда tf.keras.layers.Embedding обучается сразу со всей сеточкой, что получилось лучше

In [61]:
import pandas as pd
import numpy as np
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from sklearn.preprocessing import LabelEncoder
from stop_words import get_stop_words
from spacy.lang.ru.stop_words import STOP_WORDS
from string import punctuation
from pymorphy2 import MorphAnalyzer
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import categorical_crossentropy, SparseCategoricalCrossentropy
from navec import Navec
from tensorflow.keras.initializers import Constant

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Uliana\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [16]:
MAX_WORDS = 2000
MAX_LEN = 100

In [17]:
data = pd.read_excel('отзывы за лето.xls')
le = LabelEncoder()
data['Rating'] = le.fit_transform(data['Rating'])
data.sample(5)

,Rating,Content,Date
8204,4,Работает отлично,2017-08-02
6762,4,Спасибо огромное Сбербанку😙,2017-08-04
5858,4,Супер,2017-08-05
1645,0,"Якобы вирус нашёл! Перевод не могу совершить, ...",2017-08-11
16292,2,При осуществлении перевода не показывает сумму...,2017-07-18


In [18]:
sw = set((get_stop_words("ru") + list(STOP_WORDS))) - {'не', 'ни', 'нет'}
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt, exclude=exclude, sw=sw, morpher=morpher):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

In [19]:
data['preprocessed_text'] = data['Content'].apply(preprocess_text)

In [20]:
data.sample(5)

,Rating,Content,Date,preprocessed_text
3206,4,Все нравится. Разработчикам спасибо.,2017-08-09,нравиться разработчик
19110,3,"Каждые 5 секунд посреди экрана табличка ""в при...",2017-06-28,5 секунда экран табличка приложение сбербанк п...
4468,2,Что-то глючит,2017-08-07,чтоть глючить
1171,3,Встроенный антивирус этого не видит?,2017-08-12,встроить антивирус не видеть
19683,4,Приложением доволен. Все необходимые мне функц...,2017-06-25,приложение довольный функция выполнять работат...


In [24]:
train_corpus = " ".join(data["preprocessed_text"])
tokens = word_tokenize(train_corpus)
tokens_filtered = [word for word in tokens if word.isalnum()]
len(tokens_filtered)

102261

In [29]:
dist = FreqDist(tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(MAX_WORDS-1)]
tokens_filtered_top[:10]

['приложение',
 'не',
 'удобно',
 'работать',
 'удобный',
 'отлично',
 'нравиться',
 'хороший',
 'телефон',
 'отличный']

In [30]:
vocabulary = {v: k for k, v in enumerate(tokens_filtered_top, 1)}

In [31]:
len(vocabulary)

1999

In [32]:
def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]

In [35]:
data_train = np.asarray([text_to_sequence(text, MAX_LEN) for text in data['preprocessed_text']], dtype=np.int32)

In [38]:
X_train, X_test, y_train, y_test = train_test_split(data_train, data['Rating'], test_size=0.33)

### Embedding слой с предобученными векторами

In [66]:
path = 'navec_news_v1_1B_250K_300d_100q.tar'
emb = Navec.load(path)

In [67]:
embedding_dim = 300

embedding_matrix = np.zeros((MAX_WORDS, embedding_dim))
for word in tokens_filtered_top:
    embedding_vector = emb.get(word)
    if embedding_vector is not None:
        embedding_matrix[tokens_filtered_top.index(word)] = embedding_vector

In [68]:
embedding_matrix.shape

(2000, 300)

In [69]:
embedding_layer = Embedding(
    MAX_WORDS,
    embedding_dim,
    embeddings_initializer=Constant(embedding_matrix),
    trainable=False,
)

In [70]:
model = Sequential()
model.add(embedding_layer)
model.add(Conv1D(128, 5))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(20))
model.add(Activation("relu"))
model.add(Dropout(0.25))
model.add(Dense(5))
model.add(Activation('softmax'))

In [71]:
model.compile(loss=SparseCategoricalCrossentropy(),
              optimizer='adam',
              metrics=['accuracy'])

In [72]:
history = model.fit(X_train, y_train,
          batch_size=BATCH_SIZE,
          epochs=EPOCHES,
          verbose=0,
          validation_split=0.1)

In [73]:
score = model.evaluate(X_test, y_test, batch_size=BATCH_SIZE, verbose=1)
print('\n')
print('Test loss:', score[0])
print('Test accuracy:', score[1])

7/7 [==============================] - 4s 561ms/step - loss: 0.6954 - accuracy: 0.7567


Test loss: 0.6954426765441895
Test accuracy: 0.7566735148429871


### Embedding слой по умолчанию

In [39]:
BATCH_SIZE = 1024
EPOCHES = 10

In [44]:
model = Sequential()
model.add(Embedding(input_dim=MAX_WORDS, output_dim=128, input_length=MAX_LEN))
model.add(Conv1D(128, 5))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(20))
model.add(Activation("relu"))
model.add(Dropout(0.25))
model.add(Dense(5))
model.add(Activation('softmax'))

In [49]:
opt = Adam(learning_rate=0.001)
model.compile(loss=SparseCategoricalCrossentropy(),
              optimizer=opt,
              metrics=['accuracy'])

In [50]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 128)          256000    
                                                                 
 conv1d (Conv1D)             (None, 96, 128)           82048     
                                                                 
 activation (Activation)     (None, 96, 128)           0         
                                                                 
 global_max_pooling1d (Globa  (None, 128)              0         
 lMaxPooling1D)                                                  
                                                                 
 dense (Dense)               (None, 20)                2580      
                                                                 
 activation_1 (Activation)   (None, 20)                0         
                                                      

In [51]:
history = model.fit(X_train, y_train,
                    batch_size=BATCH_SIZE,
                    epochs=EPOCHES,
                    verbose=0,
                    validation_split=0.1)

In [52]:
score = model.evaluate(X_test, y_test, batch_size=BATCH_SIZE, verbose=1)
print('\n')
print('Test loss:', score[0])
print('Test accuracy:', score[1])

7/7 [==============================] - 2s 287ms/step - loss: 0.6603 - accuracy: 0.7697


Test loss: 0.6603455543518066
Test accuracy: 0.7697271704673767


Вывод: Результаты отличаются незначительно. С предобученными векторами модель показывает себя чуть хуже, немного больше потери - 0.7 против 0.66 и accuracy 0.76 против 0.77